In [1]:
# Main imports
from econml.dml import DML, LinearDML, SparseLinearDML, CausalForestDML

# Helper imports
import numpy as np
from itertools import product
from sklearn.linear_model import (Lasso, LassoCV, LogisticRegression,
                                  LogisticRegressionCV, LinearRegression,
                                  MultiTaskElasticNet, MultiTaskElasticNetCV)

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
import pandas as pd

from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
cities = ["Newyork", "LosAngeles", "Paris", "London", "Beijing", "Shanghai", "Guangzhou"]

In [3]:
selected_landuse = ["residential", "commercial", "retail", "brownfield", "construction", "industrial", "grass", "farmland", "forest"]

features = ['residential', 'residential_mean_distance', 'residential_mean_NTL',
            'commercial', 'commercial_mean_distance', 'commercial_mean_NTL',
            'retail', 'retail_mean_distance', 'retail_mean_NTL',
            'brownfield', 'brownfield_mean_distance', 'brownfield_mean_NTL',
            'construction', 'construction_mean_distance', 'construction_mean_NTL',
            'industrial', 'industrial_mean_distance', 'industrial_mean_NTL',
            'grass',  'grass_mean_distance', 'grass_mean_NTL',
            'farmland', 'farmland_mean_distance', 'farmland_mean_NTL',
            'forest', 'forest_mean_distance', 'forest_mean_NTL', 
            ]

indices = ["Over_illumination", "Trespass", "Clutter"]

In [7]:
df = pd.DataFrame()
for city in cities:
    tmp = pd.read_csv(f"./LightPollution/{city}_final.csv")
    tmp.rename(columns={"Tresspass": "Trespass"}, inplace = True)
    tmp = tmp[["Unnamed: 0"] + features + indices]
    tmp.reset_index(inplace = True, drop = True)
    tmp["city"] = city
    df = pd.concat([df, tmp], axis = 0)


Columns (3,4,9,11,15,31,35,37,41,46,50,51,52,59,73,82,83,103,127,132,139,142,151,152,184,189,190,192,193,254) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (5,9,11,23,28,32,34,39,46,47,55,56,57,58,59,60,61,65,66,73,81,82,83,84,94,95,96,102,128,130,131,140,144,147,149,153,158,159,161,171,209,213,217,218,227,290,294,295,296) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (1,7,11,32,34,37,40,42,48,54,55,58,63,66,72,74,78,79,82,88,89,104,152,153,155,163,176,185,191,192,211,212,213,312,330,342,343,360) have mixed types. Specify dtype option on import or set low_memory=False.


In [5]:
df[indices].describe()

,Over_illumination,Trespass,Clutter
count,35138.000000,3.509500e+04,35095.000000
mean,9593.257434,4.383913e+32,0.002089
std,11705.223309,6.283648e+33,0.001358
min,0.000000,0.000000e+00,0.000000
25%,2681.341257,1.801566e-01,0.001293
50%,5615.317506,5.854119e-01,0.002140
75%,10858.366699,1.309380e+00,0.002867
max,89335.953125,9.050201e+34,0.019711


In [28]:
df.shape

(35144, 32)

In [6]:
df.drop(df[df["Trespass"] > 10].index, axis = 0, inplace = True)
df.shape

(33642, 32)

In [11]:
df.dropna(subset = indices, inplace = True)
df.shape

(33593, 32)

In [5]:
df["Over_illumination"] = (df["Over_illumination"] - df["Over_illumination"].min()) / (df["Over_illumination"].max() - df["Over_illumination"].min())
df["Trespass"] = (df["Trespass"] - df["Trespass"].min()) / (df["Trespass"].max() - df["Trespass"].min())
df["Clutter"] = (df["Clutter"] - df["Clutter"].min()) / (df["Clutter"].max() - df["Clutter"].min())

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set_style("whitegrid")

import osmnx as ox
import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import rasterio
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

ox.config(use_cache = True, log_console = False)

In [12]:
X = df[indices].copy()

In [13]:
X["Over_illumination"] = (X["Over_illumination"] - X["Over_illumination"].min()) / (X["Over_illumination"].max() - X["Over_illumination"].min())
X["Trespass"] = (X["Trespass"] - X["Trespass"].min()) / (X["Trespass"].max() - X["Trespass"].min())
X["Clutter"] = (X["Clutter"] - X["Clutter"].min()) / (X["Clutter"].max() - X["Clutter"].min())

In [14]:
kmeans = KMeans(n_clusters = 4)
kmeans.fit(X)

y_kmeans = kmeans.predict(X)

kmeans = KMeans(n_clusters = 4)
kmeans.fit(X)

y_kmeans = kmeans.predict(X)

In [15]:
kmeans.cluster_centers_

array([[0.04747507, 0.03117508, 0.07855411],
       [0.54157506, 0.63875368, 0.18793869],
       [0.33317272, 0.37379794, 0.16011303],
       [0.12831433, 0.13077465, 0.13441341]])

In [16]:
df["level"] = y_kmeans
df["level"] = df["level"].map({0: 0, 3: 1, 2: 2, 1: 3})
df.groupby("city")["level"].value_counts()

city        level
Beijing     0        4430
            1        1838
            2          50
Guangzhou   0        3142
            1         727
London      0        1948
            1        1834
            3        1684
            2        1001
LosAngeles  0        4663
            1         737
            2         209
Newyork     1        2012
            0         600
            2          75
            3          24
Paris       2          57
            3          56
            1          31
Shanghai    0        5307
            1        2418
            2         708
            3          42
Name: level, dtype: int64

In [21]:
df[df["city"] == "Beijing"]

,Unnamed: 0,residential,residential_mean_distance,residential_mean_NTL,commercial,commercial_mean_distance,commercial_mean_NTL,retail,retail_mean_distance,retail_mean_NTL,...,farmland_mean_distance,farmland_mean_NTL,forest,forest_mean_distance,forest_mean_NTL,Over_illumination,Trespass,Clutter,city,level
5,5,101.0,1497.825272,41.372574,30.0,1788.690559,47.559500,3.0,2119.816969,42.566667,...,NaN,NaN,11.0,1978.706956,42.401818,8493.130015,1.261739,0.002905,Beijing,1
6,7,59.0,1666.997805,34.834576,10.0,1472.100068,34.936000,2.0,1229.927398,38.037501,...,NaN,NaN,2.0,2529.698356,27.907500,3246.340013,0.482401,0.002460,Beijing,0
7,8,100.0,1459.343427,41.697000,30.0,1650.207917,47.391833,4.0,2219.164112,50.628750,...,NaN,NaN,9.0,2165.881517,43.500556,8822.995026,1.310459,0.002933,Beijing,1
8,9,31.0,1695.903925,31.130483,1.0,892.743352,35.754997,0.0,NaN,NaN,...,NaN,NaN,1.0,2738.913359,10.260000,2247.914974,0.319818,0.001893,Beijing,0
10,11,43.0,1473.517866,51.500698,20.0,1781.152954,64.454000,3.0,2217.076691,58.593333,...,NaN,NaN,7.0,1896.866961,62.292143,9398.445022,1.468675,0.003654,Beijing,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6585,7798,44.0,1938.450656,37.944091,16.0,1828.352697,41.983124,2.0,1933.537537,39.009999,...,NaN,NaN,2.0,2118.227643,38.077500,3711.674964,0.431683,0.002298,Beijing,0
6586,7799,59.0,1841.094139,40.418051,0.0,NaN,NaN,1.0,2529.726264,51.160000,...,NaN,NaN,3.0,2458.762503,32.136667,3435.369988,0.399540,0.002372,Beijing,0
6587,7800,31.0,1832.878795,36.675000,0.0,NaN,NaN,3.0,2212.858168,28.085000,...,NaN,NaN,2.0,1086.503264,23.357499,2426.200000,0.277214,0.001846,Beijing,0
6588,7801,20.0,1938.181312,14.282750,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,351.839999,0.042222,0.001051,Beijing,0


In [26]:
df.to_csv("result.csv")

In [5]:
df = pd.read_csv("result.csv")
df

,Unnamed: 0.1,Unnamed: 0,residential,residential_mean_distance,residential_mean_NTL,commercial,commercial_mean_distance,commercial_mean_NTL,retail,retail_mean_distance,...,farmland_mean_distance,farmland_mean_NTL,forest,forest_mean_distance,forest_mean_NTL,Over_illumination,Trespass,Clutter,city,level
0,5,5,25.0,1689.161715,65.130203,0.0,NaN,NaN,2.0,2171.989339,...,NaN,NaN,0.0,NaN,NaN,24525.742188,3.221751,0.003010,Newyork,2
1,6,6,9.0,1085.097301,77.835556,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,17134.656250,2.822740,0.008588,Newyork,2
2,7,7,9.0,1074.091469,77.835556,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,17252.503906,2.846051,0.008585,Newyork,2
3,8,8,25.0,1678.305704,66.441994,0.0,NaN,NaN,1.0,2579.055915,...,NaN,NaN,0.0,NaN,NaN,20412.195312,2.910449,0.002674,Newyork,2
4,10,10,7.0,1143.017269,60.984283,0.0,NaN,NaN,1.0,2229.745795,...,NaN,NaN,7.0,2170.985952,53.627140,2357.625488,0.314219,0.005585,Newyork,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33588,4100,4100,100.0,1738.222404,49.516950,2.0,2002.573122,39.507499,3.0,1221.041800,...,NaN,NaN,5.0,1548.174338,48.326000,8873.589923,1.192334,0.003838,Guangzhou,1
33589,4101,4101,50.0,2134.720596,39.951200,2.0,2528.869960,43.892500,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,3772.600035,0.463571,0.002757,Guangzhou,0
33590,4102,4102,36.0,1916.519761,31.831944,11.0,1147.403349,30.699091,6.0,1040.984588,...,NaN,NaN,4.0,1901.347961,13.691251,2530.920017,0.393705,0.002317,Guangzhou,0
33591,4103,4103,16.0,2010.072860,38.020313,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,633.460004,0.070655,0.003712,Guangzhou,0


In [6]:
df["score"] = df["Over_illumination"] + df["Trespass"] + df["Clutter"]
df.groupby("city").mean()["score"]

city
Beijing       0.249566
Guangzhou     0.202481
London        0.647397
LosAngeles    0.163970
Newyork       0.394836
Paris         1.005815
Shanghai      0.300971
Name: score, dtype: float64

In [39]:
for landuse in selected_landuse:
    df.loc[df[landuse] == 0, landuse + "_mean_distance"] = 3000
    df.loc[df[landuse] == 0, landuse + "_mean_NTL"] = 0

In [40]:
df

,Unnamed: 0.1,Unnamed: 0,residential,residential_mean_distance,residential_mean_NTL,commercial,commercial_mean_distance,commercial_mean_NTL,retail,retail_mean_distance,...,farmland_mean_distance,farmland_mean_NTL,forest,forest_mean_distance,forest_mean_NTL,Over_illumination,Trespass,Clutter,city,level
0,5,5,25.0,1689.161715,65.130203,0.0,3000.000000,0.000000,2.0,2171.989339,...,3000.0,0.0,0.0,3000.000000,0.000000,24525.742188,3.221751,0.003010,Newyork,2
1,6,6,9.0,1085.097301,77.835556,0.0,3000.000000,0.000000,0.0,3000.000000,...,3000.0,0.0,0.0,3000.000000,0.000000,17134.656250,2.822740,0.008588,Newyork,2
2,7,7,9.0,1074.091469,77.835556,0.0,3000.000000,0.000000,0.0,3000.000000,...,3000.0,0.0,0.0,3000.000000,0.000000,17252.503906,2.846051,0.008585,Newyork,2
3,8,8,25.0,1678.305704,66.441994,0.0,3000.000000,0.000000,1.0,2579.055915,...,3000.0,0.0,0.0,3000.000000,0.000000,20412.195312,2.910449,0.002674,Newyork,2
4,10,10,7.0,1143.017269,60.984283,0.0,3000.000000,0.000000,1.0,2229.745795,...,3000.0,0.0,7.0,2170.985952,53.627140,2357.625488,0.314219,0.005585,Newyork,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33588,4100,4100,100.0,1738.222404,49.516950,2.0,2002.573122,39.507499,3.0,1221.041800,...,3000.0,0.0,5.0,1548.174338,48.326000,8873.589923,1.192334,0.003838,Guangzhou,1
33589,4101,4101,50.0,2134.720596,39.951200,2.0,2528.869960,43.892500,0.0,3000.000000,...,3000.0,0.0,0.0,3000.000000,0.000000,3772.600035,0.463571,0.002757,Guangzhou,0
33590,4102,4102,36.0,1916.519761,31.831944,11.0,1147.403349,30.699091,6.0,1040.984588,...,3000.0,0.0,4.0,1901.347961,13.691251,2530.920017,0.393705,0.002317,Guangzhou,0
33591,4103,4103,16.0,2010.072860,38.020313,0.0,3000.000000,0.000000,0.0,3000.000000,...,3000.0,0.0,0.0,3000.000000,0.000000,633.460004,0.070655,0.003712,Guangzhou,0
